NUOVO

In [1]:
from IPython.display import clear_output, display
import ipywidgets as widgets
from datetime import datetime, timedelta
import os
import torch

# 📁 Percorso modello
MODEL_PATH = "/content/drive/MyDrive/NeuroOnco/Modello/cnn_model_with_series.pt"

# 🔔 Output informativo
msg_out = widgets.Output()
with msg_out:
    clear_output()
    print("🔽 Seleziona un paziente per iniziare...")

with msg_out:
    clear_output()
    print("📊 Informazioni sul modello CNN:")
    try:
        model_info = torch.load(MODEL_PATH, map_location='cpu')
        selected_series = model_info.get("selected_series", [])
        model_time = os.path.getmtime(MODEL_PATH)
        dt = datetime.fromtimestamp(model_time) + timedelta(hours=2)  # 🇮🇹 Ora locale
        model_time_str = dt.strftime("%Y-%m-%d %H:%M:%S")

        print(f"📦 File modello: {os.path.basename(MODEL_PATH)}")
        print(f"📅 Salvato il: {model_time_str}")
        print(f"🧠 Serie usate per l’addestramento: {selected_series}")
    except Exception as e:
        print(f"❌ Errore nel caricamento del modello: {e}")



In [2]:
import os
import ipywidgets as widgets
from IPython.display import display

# 📁 Cartella con i file .pt
PT_DIR = "/content/drive/MyDrive/NeuroOnco/Derivate"

# 🔍 Estrai nomi file senza estensione
pt_files = sorted([f[:-3] for f in os.listdir(PT_DIR) if f.endswith(".pt")])




In [3]:
import matplotlib.pyplot as plt

colori_roi = {
    'SANO': 'green',
    'MALATO': 'red',
    'EXTERNAL': 'blue'
}

# Colori per ROI aggiuntive
from itertools import cycle
altre_colorazioni = cycle(plt.cm.tab10.colors)

# Aggiunge colore casuale per ogni ROI nuova non già mappata
def colore_per_roi(nome_roi):
    if nome_roi not in colori_roi:
        colori_roi[nome_roi] = next(altre_colorazioni)
    return colori_roi[nome_roi]


In [4]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from itertools import cycle
from datetime import datetime, timedelta  # Assicurati di avere anche questo import

# 📁 Cartella con i file .pt
PT_DIR = "/content/drive/MyDrive/NeuroOnco/Derivate"
pt_files = sorted([f[:-3] for f in os.listdir(PT_DIR) if f.endswith(".pt")])

# 🧾 Output messaggi
msg_out = widgets.Output()
with msg_out:
    clear_output()
    print("🔽 Seleziona un paziente per iniziare...")

# 🔽 Dropdown con valore iniziale neutro
dropdown_paziente = widgets.Dropdown(
    options=["— Seleziona —"] + pt_files,
    value="— Seleziona —",
    description="Paziente"
)

# 🎨 Colori per ROI
colori_roi = {'SANO': 'green', 'MALATO': 'red', 'EXTERNAL': 'blue'}
altre_colorazioni = cycle(plt.cm.tab10.colors)

def colore_per_roi(nome_roi):
    if nome_roi not in colori_roi:
        colori_roi[nome_roi] = next(altre_colorazioni)
    return colori_roi[nome_roi]

def carica_mappa_probabilita(pt_file_name):
    base_name = os.path.splitext(pt_file_name)[0]
    for suffix in ["_probs.npy", "_prob_map.npy"]:
        prob_path = os.path.join(PT_DIR, base_name + suffix)
        if os.path.exists(prob_path):
            return np.load(prob_path), prob_path
    return None, None

# 👁️‍🗨️ Visualizzatore interattivo
def visualizza_interattivo(file_pt):
    with msg_out:
        clear_output()
        print("⏳ Caricamento volume...")

    if not file_pt.endswith(".pt"):
        file_pt += ".pt"

    path = os.path.join(PT_DIR, file_pt)
    data = torch.load(path, map_location=torch.device('cpu'))
    vol = data['volume'].numpy()
    rois = data['roi_masks']
    nomi_serie = data.get('nomi_serie', [f"Serie {i}" for i in range(vol.shape[0])])
    prob_map, _ = carica_mappa_probabilita(file_pt)

    with msg_out:
        clear_output()
        file_path = os.path.join(PT_DIR, file_pt)
        dt = datetime.fromtimestamp(os.path.getmtime(file_path)) + timedelta(hours=2)  # 🇮🇹 fuso orario
        data_str = dt.strftime("%Y-%m-%d %H:%M:%S")
        print(f"✅ Paziente caricato: {file_pt}")
        print(f"📅 Ultima modifica: {data_str}")

    s_serie = widgets.Dropdown(options=[(nome, idx) for idx, nome in enumerate(nomi_serie)], description="Serie")
    s_z = widgets.IntSlider(description="Slice Z")
    s_prob = widgets.FloatSlider(min=0, max=1, step=0.01, description="Soglia Prob", value=0.0)
    w_roi = widgets.SelectMultiple(options=list(rois.keys()), description=" ROI", layout=widgets.Layout(height='100px'))
    w_prob_on = widgets.Checkbox(value=False, description="Mostra Probabilità")
    w_minmax = widgets.FloatRangeSlider(description="Window", step=0.01, continuous_update=False)
    w_scope_hist = widgets.ToggleButtons(options=['Slice', 'Volume'], description='Istogramma:', value='Slice')

    def aggiorna_range_z(*args):
        zmax = vol[s_serie.value].shape[0] - 1
        s_z.max = zmax
        s_z.value = min(s_z.value, zmax)
        vol_valid = vol[s_serie.value][np.isfinite(vol[s_serie.value])]
        vol_valid = vol_valid[vol_valid > -999]
        vmin, vmax = np.percentile(vol_valid, [1, 99])
        w_minmax.min = float(vol_valid.min())
        w_minmax.max = float(vol_valid.max())
        w_minmax.value = [vmin, vmax]

    s_serie.observe(aggiorna_range_z, names='value')
    aggiorna_range_z()

    def aggiorna(serie, z, soglia, roi_sel, show_prob, window, scope_hist):
        fig = plt.figure(figsize=(12, 6))
        ax1 = fig.add_subplot(121)
        img = np.clip(vol[serie, z], window[0], window[1])
        ax1.imshow(img, cmap='gray')
        ax1.set_title(f"Slice {z} | Serie: {nomi_serie[serie]}")
        ax1.axis('off')

        if show_prob and prob_map is not None and z < prob_map.shape[0]:
            prob_slice = prob_map[z]
            ax1.imshow(np.where(prob_slice >= soglia, prob_slice, np.nan), cmap='magma', alpha=0.5)

        for roi_name in roi_sel:
            if roi_name not in rois or z >= rois[roi_name].shape[0]:
                continue
            mask = rois[roi_name][z].numpy().astype(bool)
            colore = colore_per_roi(roi_name)
            ax1.contour(mask, colors=[colore], linewidths=1)

        ax2 = fig.add_subplot(122)
        ax2.set_title("Istogramma ROI selezionate")
        ax2.set_xlabel("Intensità (normalizzato)")
        ax2.set_ylabel("Densità")

        all_values = []
        for roi_name in roi_sel:
            if roi_name not in rois:
                continue

            if scope_hist == 'Slice':
                if z >= rois[roi_name].shape[0]:
                    continue
                mask = rois[roi_name][z].numpy().astype(bool)
                values = vol[serie, z][mask]
                values = values[values > -1000]
            else:
                mask = rois[roi_name].numpy().astype(bool)
                values = vol[serie][mask]

            if len(values) > 0:
                all_values.append(values)

            if all_values:
                filtered_values = [v[v > -1000] for v in all_values if len(v[v > -1000]) > 0]

                if not filtered_values:
                    ax2.text(0.5, 0.5, "Tutti i valori filtrati (< -1000)", ha='center', va='center', transform=ax2.transAxes)
                else:
                    all_concat = np.concatenate(filtered_values)
                    vmin, vmax = np.percentile(all_concat, [1, 99])
                    bins = np.linspace(vmin, vmax, 50)

                    for roi_name in roi_sel:
                        if roi_name not in rois:
                            continue

                        if scope_hist == 'Slice':
                            if z >= rois[roi_name].shape[0]:
                                continue
                            mask = rois[roi_name][z].numpy().astype(bool)
                            values = vol[serie, z][mask]
                        else:
                            mask = rois[roi_name].numpy().astype(bool)
                            values = vol[serie][mask]

                        values = values[values > -1000]  # 🔍 importante
                        if len(values) > 0:
                            colore = colore_per_roi(roi_name)
                            ax2.hist(values, bins=bins, alpha=0.3, label=roi_name, color=colore, density=True)

                    ax2.legend()

                    vmin, vmax = np.percentile(all_values, [1, 99])
                    bins = np.linspace(vmin, vmax, 50)

                    for roi_name in roi_sel:
                        if roi_name not in rois:
                            continue

                        if scope_hist == 'Slice':
                            if z >= rois[roi_name].shape[0]:
                                continue
                            mask = rois[roi_name][z].numpy().astype(bool)
                            values = vol[serie, z][mask]
                        else:
                            mask = rois[roi_name].numpy().astype(bool)
                            values = vol[serie][mask]

                        values = values[values > -1000]  # 🔍 importante
                        if len(values) > 0:
                            colore = colore_per_roi(roi_name)
                            ax2.hist(values, bins=bins, alpha=0.3, label=roi_name, color=colore, density=True)

                    ax2.legend()

        else:
            ax2.text(0.5, 0.5, "Nessuna ROI selezionata", ha='center', va='center', transform=ax2.transAxes)

        plt.tight_layout()
        plt.show()

    display(widgets.VBox([
        s_serie, s_z, w_minmax,
        s_prob, w_prob_on,
        w_roi, w_scope_hist,
        widgets.interactive_output(
            aggiorna,
            {
                'serie': s_serie,
                'z': s_z,
                'soglia': s_prob,
                'roi_sel': w_roi,
                'show_prob': w_prob_on,
                'window': w_minmax,
                'scope_hist': w_scope_hist
            }
        )
    ]))


# 🔄 Callback per selezione del paziente
def on_paziente_cambiato(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if change['new'] != "— Seleziona —":
            visualizza_interattivo(change['new'])

# 🔗 Collega il callback al dropdown
dropdown_paziente.observe(on_paziente_cambiato, names='value')

# 📺 Mostra interfaccia
display(widgets.VBox([dropdown_paziente, msg_out]))

MODIFICA

In [5]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from itertools import cycle

# Percorso ai dati
PT_DIR = "/content/drive/MyDrive/NeuroOnco/Derivate"
pt_files = sorted([f for f in os.listdir(PT_DIR) if f.endswith(".pt")])

# Colori ROI
colori_roi = {'SANO': 'green', 'MALATO': 'red', 'EXTERNAL': 'blue'}
altre_colorazioni = cycle(plt.cm.tab10.colors)
def colore_per_roi(nome_roi):
    if nome_roi not in colori_roi:
        colori_roi[nome_roi] = next(altre_colorazioni)
    return colori_roi[nome_roi]

from datetime import datetime, timedelta


# Funzione visualizzazione immagini
def visualizza_slice(file_pt):
    print("⏳ Caricamento volume...")

    file_path = os.path.join(PT_DIR, file_pt)
    dt = datetime.fromtimestamp(os.path.getmtime(file_path)) + timedelta(hours=2)
    data_str = dt.strftime("%Y-%m-%d %H:%M:%S")
    print(f"✅ File caricato: {file_pt}")
    print(f"📅 Ultima modifica: {data_str}")

    path = os.path.join(PT_DIR, file_pt)
    data = torch.load(path, map_location='cpu')
    vol = data["volume"].numpy()
    rois = data["roi_masks"]
    nomi_serie = data.get("nomi_serie", [f"Serie {i}" for i in range(vol.shape[0])])

    s_serie = widgets.Dropdown(options=[(n, i) for i, n in enumerate(nomi_serie)], description="Serie")
    s_z = widgets.IntSlider(description="Slice Z")
    w_roi = widgets.SelectMultiple(options=list(rois.keys()), description="ROI", layout=widgets.Layout(height='100px'))
    w_minmax = widgets.FloatRangeSlider(description="Window", step=0.01)

    def aggiorna_range(*args):
        serie = s_serie.value
        zmax = vol[serie].shape[0] - 1
        s_z.max = zmax
        vol_valid = vol[serie][np.isfinite(vol[serie]) & (vol[serie] > -999)]
        vmin, vmax = np.percentile(vol_valid, [1, 99])
        w_minmax.min = float(vol_valid.min())
        w_minmax.max = float(vol_valid.max())
        w_minmax.value = [vmin, vmax]

    s_serie.observe(aggiorna_range, names='value')
    aggiorna_range()

    def aggiorna(serie, z, roi_sel, window):
        plt.figure(figsize=(6, 6))
        img = np.clip(vol[serie, z], window[0], window[1])
        plt.imshow(img, cmap='gray')
        plt.title(f"Slice {z} | Serie: {nomi_serie[serie]}")
        plt.axis('off')
        for roi_name in roi_sel:
            if roi_name not in rois or z >= rois[roi_name].shape[0]:
                continue
            mask = rois[roi_name][z].numpy().astype(bool)
            plt.contour(mask, colors=[colore_per_roi(roi_name)], linewidths=1)
        plt.show()

    display(widgets.VBox([
        s_serie, s_z, w_minmax, w_roi,
        widgets.interactive_output(aggiorna, {
            "serie": s_serie, "z": s_z,
            "roi_sel": w_roi, "window": w_minmax
        })
    ]))

# Dropdown pazienti
dropdown_pt = widgets.Dropdown(options=["— Seleziona —"] + pt_files, description="Paziente")

def on_change(change):
    if change['new'] != "— Seleziona —":
        clear_output()
        visualizza_slice(change['new'])

dropdown_pt.observe(on_change, names='value')
display(dropdown_pt)

Dropdown(description='Paziente', options=('— Seleziona —', 'ID1.pt', 'ID2.pt', 'ID3.pt', 'ID5.pt', 'ID6.pt', '…

In [12]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from itertools import cycle

# Percorso ai dati
PT_DIR = "/content/drive/MyDrive/NeuroOnco/Derivate"
pt_files = sorted([f for f in os.listdir(PT_DIR) if f.endswith(".pt")])

# Colori ROI
colori_roi = {'SANO': 'green', 'MALATO': 'red', 'EXTERNAL': 'blue'}
altre_colorazioni = cycle(plt.cm.tab10.colors)
def colore_per_roi(nome_roi):
    if nome_roi not in colori_roi:
        colori_roi[nome_roi] = next(altre_colorazioni)
    return colori_roi[nome_roi]

# Funzione istogramma
def visualizza_istogramma_roi(file_pt):
    print("⏳ Caricamento volume...")

    file_path = os.path.join(PT_DIR, file_pt)
    dt = datetime.fromtimestamp(os.path.getmtime(file_path)) + timedelta(hours=2)
    data_str = dt.strftime("%Y-%m-%d %H:%M:%S")
    print(f"✅ File caricato: {file_pt}")
    print(f"📅 Ultima modifica: {data_str}")

    path = os.path.join(PT_DIR, file_pt)
    data = torch.load(path, map_location='cpu')
    ...

    vol = data["volume"].numpy()
    rois = data["roi_masks"]
    nomi_serie = data.get("nomi_serie", [f"Serie {i}" for i in range(vol.shape[0])])

    s_serie = widgets.Dropdown(options=[(n, i) for i, n in enumerate(nomi_serie)], description="Serie")
    s_z = widgets.IntSlider(description="Slice Z")
    w_roi = widgets.SelectMultiple(options=list(rois.keys()), description="ROI", layout=widgets.Layout(height='100px'))
    w_scope = widgets.ToggleButtons(options=["Slice", "Volume"], description="Ambito")

    def aggiorna(serie, z, roi_sel, scope):
        all_values = []
        for roi_name in roi_sel:
            if roi_name not in rois:
                continue
            if scope == "Slice":
                if z >= rois[roi_name].shape[0]:
                    continue
                mask = rois[roi_name][z].numpy().astype(bool)
                values = vol[serie, z][mask]
            else:
                mask = rois[roi_name].numpy().astype(bool)
                values = vol[serie][mask]
            values = values[np.isfinite(values) & (values > -1000)]
            if len(values) > 0:
                all_values.append((roi_name, values))

        if not all_values:
            print("⚠️ Nessun dato valido.")
            return

        plt.figure(figsize=(8, 5))
        for roi_name, values in all_values:
            colore = colore_per_roi(roi_name)
            plt.hist(values, bins=50, alpha=0.3, label=roi_name, color=colore, density=True)
        plt.xlabel("Intensità")
        plt.ylabel("Densità")
        plt.title(f"Istogramma {scope.lower()} delle ROI")
        plt.legend()
        plt.tight_layout()
        plt.show()

    display(widgets.VBox([
        s_serie, s_z, w_scope, w_roi,
        widgets.interactive_output(aggiorna, {
            "serie": s_serie, "z": s_z,
            "roi_sel": w_roi, "scope": w_scope
        })
    ]))

# Dropdown pazienti
dropdown_pt = widgets.Dropdown(options=["— Seleziona —"] + pt_files, description="Paziente")

def on_change_hist(change):
    if change['new'] != "— Seleziona —":
        clear_output()
        visualizza_istogramma_roi(change['new'])

dropdown_pt.observe(on_change_hist, names='value')
display(dropdown_pt)

⏳ Caricamento volume...
✅ File caricato: ID2.pt
📅 Ultima modifica: 2025-10-06 13:49:08


In [7]:
def visualizza_istogramma_roi_multiplo_volume():
    pt_files = sorted([f for f in os.listdir(PT_DIR) if f.endswith(".pt")])
    pazienti_sel = widgets.SelectMultiple(options=pt_files, description="Pazienti", layout=widgets.Layout(height='120px'))
    w_roi = widgets.SelectMultiple(description="ROI", layout=widgets.Layout(height='100px'))
    s_serie = widgets.Dropdown(description="Serie")
    output_log = widgets.Output()

    # Funzione per aggiornare ROI e serie quando cambia il paziente selezionato
    def aggiorna_roi_lista(change=None):
        if pazienti_sel.value:
            path = os.path.join(PT_DIR, pazienti_sel.value[0])
            data = torch.load(path, map_location='cpu')
            roi_keys = list(data.get("roi_masks", {}).keys())
            nomi_serie = data.get("nomi_serie", [f"Serie {i}" for i in range(data['volume'].shape[0])])
            w_roi.options = roi_keys
            s_serie.options = [(n, i) for i, n in enumerate(nomi_serie)]
            if s_serie.value is None and len(nomi_serie) > 0:
                s_serie.value = 0

    pazienti_sel.observe(aggiorna_roi_lista, names='value')
    s_serie.observe(aggiorna_roi_lista, names='value')

    # Funzione principale per aggiornare l'istogramma
    def aggiorna(pazienti, roi_sel, serie):
        with output_log:
            clear_output()
            if not pazienti or not roi_sel:
                print("⚠️ Seleziona almeno un paziente e una ROI.")
                return

            print(f"📊 Caricamento e analisi di {len(pazienti)} pazienti...")
            plt.figure(figsize=(10, 6))

            for pt in pazienti:
                try:
                    path = os.path.join(PT_DIR, pt)
                    data = torch.load(path, map_location='cpu')
                    vol = data["volume"].numpy()
                    rois = data["roi_masks"]

                    for roi_name in roi_sel:
                        if roi_name not in rois:
                            continue
                        mask = rois[roi_name].numpy().astype(bool)
                        values = vol[serie][mask]
                        values = values[np.isfinite(values) & (values > -1000)]
                        if len(values) > 0:
                            label = f"{pt[:-3]} - {roi_name}"
                            colore = colore_per_roi(label)
                            plt.hist(values, bins=50, alpha=0.3, label=label, color=colore, density=True)

                except Exception as e:
                    print(f"❌ Errore con {pt}: {e}")

            plt.title("Istogramma delle ROI su tutto il volume")
            plt.xlabel("Intensità")
            plt.ylabel("Densità")
            plt.legend(fontsize=8, loc='upper right')
            plt.tight_layout()
            plt.show()
            print("✅ Completato.")

    display(widgets.VBox([
        pazienti_sel, w_roi, s_serie,
        widgets.interactive_output(aggiorna, {
            "pazienti": pazienti_sel,
            "roi_sel": w_roi,
            "serie": s_serie
        }),
        output_log
    ]))

In [11]:
visualizza_istogramma_roi_multiplo_volume()

In [9]:
from IPython.display import clear_output, display
import torch, numpy as np, os, matplotlib.pyplot as plt, ipywidgets as widgets

def crea_speculare_con_interazione(cartella, nome_roi='MALATO_COM'):
    pt_files = [f for f in os.listdir(cartella) if f.endswith('.pt')]
    idx = 0
    shift_y = 0
    shift_x = 0

    files_sel = widgets.SelectMultiple(
        options=pt_files, description="Pazienti",
        layout=widgets.Layout(width='60%', height='150px'))
    btn_start = widgets.Button(description="▶️ Avvia", button_style='primary')
    txt_y = widgets.Text(description="Shift Y", placeholder="es. -20 o 10")
    txt_x = widgets.Text(description="Shift X", placeholder="es. 5 o -5")
    btn_apply = widgets.Button(description="Applica shift", button_style='info')
    btn_save = widgets.Button(description="✅ Salva ROI speculare", button_style='success')
    out = widgets.Output()

    def mostra():
        nonlocal shift_x, shift_y, idx
        with out:
            clear_output()
            if idx >= len(files_sel.value):
                print("✅ Tutti i pazienti elaborati.")
                return
            fn = files_sel.value[idx]
            print(f"⚙️ Paziente {idx+1}/{len(files_sel.value)}: {fn}")
            data = torch.load(os.path.join(cartella, fn), map_location='cpu')
            vol = data["volume"][0].numpy()
            roi = data["roi_masks"].get(nome_roi)
            if roi is None:
                print(f"❗ ROI {nome_roi} non trovata!")
                idx += 1
                return

            # calcola slice z centrale della ROI
            nonzero_z = torch.any(roi, dim=(1,2)).nonzero().squeeze()
            if nonzero_z.numel() == 0:
                print("❗ ROI vuota!")
                idx += 1
                return
            zc = nonzero_z[len(nonzero_z)//2].item()

            orig = roi[zc].numpy()
            spec = np.fliplr(orig)

            def shift_arr(arr):
                dst = np.zeros_like(arr)
                sy, sx = shift_y, shift_x
                y1, x1 = max(0, sy), max(0, sx)
                y2 = min(arr.shape[0], arr.shape[0] + sy)
                x2 = min(arr.shape[1], arr.shape[1] + sx)
                sy1, sx1 = max(0, -sy), max(0, -sx)
                sy2 = sy1 + (y2 - y1)
                sx2 = sx1 + (x2 - x1)
                dst[y1:y2, x1:x2] = arr[sy1:sy2, sx1:sx2]
                return dst

            spec_s = shift_arr(spec)
            plt.figure(figsize=(6,6))
            plt.imshow(vol[zc], cmap='gray')
            plt.contour(orig, colors='red', linewidths=1)
            plt.contour(spec_s, colors='lime', linewidths=1)
            plt.title(f"{fn} | slice {zc} | Y={shift_y} X={shift_x}")
            plt.axis('off')
            plt.show()

    def on_start(_):
        nonlocal idx, shift_x, shift_y
        idx = 0; shift_x = 0; shift_y = 0
        mostra()

    def on_apply(_):
        nonlocal shift_x, shift_y
        try:
            shift_y = int(txt_y.value)
            shift_x = int(txt_x.value)
        except ValueError:
            with out:
                print("❗ Inserisci un numero intero valido in Y e X.")
            return
        mostra()

    def on_save(_):
        nonlocal idx, shift_x, shift_y
        fn = files_sel.value[idx]
        data = torch.load(os.path.join(cartella, fn), map_location='cpu')
        roi = data["roi_masks"][nome_roi]
        spec = torch.flip(roi, dims=[2])
        roi_shift = torch.zeros_like(spec)
        nonzero_z = torch.any(roi, dim=(1,2)).nonzero().squeeze()
        zc = nonzero_z[len(nonzero_z)//2].item()

        def shift_arr(arr):
            dst = np.zeros_like(arr)
            sy, sx = shift_y, shift_x
            y1, x1 = max(0, sy), max(0, sx)
            y2 = min(arr.shape[0], arr.shape[0] + sy)
            x2 = min(arr.shape[1], arr.shape[1] + sx)
            sy1, sx1 = max(0, -sy), max(0, -sx)
            sy2 = sy1 + (y2 - y1)
            sx2 = sx1 + (x2 - x1)
            dst[y1:y2, x1:x2] = arr[sy1:sy2, sx1:sx2]
            return dst

        for z in range(spec.shape[0]):
            arr = spec[z].numpy()
            roi_shift[z] = torch.tensor(shift_arr(arr), dtype=torch.uint8)

        data["roi_masks"][f"{nome_roi}_SPEC"] = roi_shift
        torch.save(data, os.path.join(cartella, fn))
        idx += 1; shift_y = 0; shift_x = 0
        txt_y.value = txt_x.value = ''
        mostra()

    btn_start.on_click(on_start)
    btn_apply.on_click(on_apply)
    btn_save.on_click(on_save)

    display(widgets.VBox([
        files_sel,
        btn_start,
        widgets.HBox([txt_y, txt_x, btn_apply]),
        btn_save,
        out
    ]))


# Chiamata
crea_speculare_con_interazione('/content/drive/MyDrive/NeuroOnco/Derivate')
